In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
# ───────── CONFIG ─────────
IMG_HEIGHT, IMG_WIDTH = 224, 224
CSV_PATH = r"C:\Users\tejas\Downloads\archive (1)\Licplatesdetection_train.csv"
IMAGE_DIR = r"C:\Users\tejas\Downloads\archive (1)\Licplatesrecognition_train\license_plates_recognition_train"

In [ ]:
# ───────── LOAD DATA ─────────
print("Loading detection dataset …")
df = pd.read_csv(CSV_PATH)

In [ ]:
images, boxes = [], []
for _, row in df.iterrows():
    img_path = os.path.join(IMAGE_DIR, row["img_id"])
    img = cv2.imread(img_path)
    if img is None:
        continue
    h, w = img.shape[:2]

    # Resize image and normalize bbox
    img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img_resized = img_resized.astype("float32") / 255.0

    ymin = row["ymin"] / h
    xmin = row["xmin"] / w
    ymax = row["ymax"] / h
    xmax = row["xmax"] / w

    images.append(img_resized)
    boxes.append([ymin, xmin, ymax, xmax])

In [ ]:
X = np.array(images, dtype="float32")
y = np.array(boxes, dtype="float32")
print("Shape:", X.shape, y.shape)

In [ ]:
# ───────── MODEL ─────────
def build_detector_model():
    base = tf.keras.applications.MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                              include_top=False, weights='imagenet', pooling='avg')
    x = layers.Dense(128, activation="relu")(base.output)
    x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(4, activation="sigmoid")(x)  # 4 coords: ymin, xmin, ymax, xmax
    return models.Model(inputs=base.input, outputs=out)

In [ ]:
model = build_detector_model()
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

In [ ]:
# ───────── TRAINING ─────────
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_tr, y_tr, validation_data=(X_val, y_val), epochs=20, batch_size=32)

In [ ]:
# ───────── SAVE MODEL ─────────
model.save("license_plate_detector.keras")
print("[ok] Detection model saved as license_plate_detector.keras")